### Imports

In [28]:
from composable import pipeable
from composable.strict import map
import composable_records as rec
from composable_glob import glob
from composable_utility import with_open
import csv
from functools import reduce as base_reduce

# --- Custom pipeable functions ---

In [29]:
# --- Custom pipeable functions ---
@pipeable
def fold(update, init, seq):
    return base_reduce(update, seq, init)

@pipeable
def unfoldr(empty, func, state):
    out = []
    while not empty(state):
        new, state = func(state)
        out = out + [new]
    return out


# **Paths and course-related functions**

In [30]:
# **Paths and course-related functions**
paths = (
    "./attendance_example_fixed_width/*/*.txt"
    >> glob(recursive=True)
)

normalize_path = lambda path: path.replace("\\", "/")
get_course = lambda path: normalize_path(path).split("/")[-2][:-2]
get_section = lambda path: get_course(path)[-2:]
get_type = lambda path: "Attendance" if "Attendance Quiz" in path else "Practice Quiz"

# --- Attendance and Practice Quiz fixed-width column spec ---

In [31]:
# --- Attendance and Practice Quiz fixed-width column spec ---
attendance_header = [
    ("Org Defined ID", 9),
    ("UserName", 9),
    ("FirstName", 12),
    ("LastName", 12),
    ("Attempt #", 3),
    ("Score", 3),
    ("Out Of", 3),
    ("Attempt_Start", 10),     # Date
    ("Attempt_Start_Time", 10),
    ("Attempt_End", 10),       # Date
    ("Attempt_End_Time", 10),
    ("Percent", 6),
]
column_names = [col[0] for col in attendance_header]
column_widths = [col[1] for col in attendance_header]

# --- Function to parse one line using unfoldr ---

In [32]:
# --- Function to parse one line using unfoldr ---
def parse_fixed_width_unfold(line, widths):
    return unfoldr(
        empty = lambda s: s == "",
        func = lambda s: (s[:widths[0]].strip(), s[widths[0]:]) if s else ("", ""),
        state = line
    )

# --- Composable pipeline to combine attendance and practice quiz files ---

In [35]:
# --- Composable pipeline to combine attendance and practice quiz files ---
combined_records = (
    paths
    >> map(
        rec.create(
            path    = lambda p: p,
            course  = lambda p: get_course(p),
            section = lambda p: get_section(p),
            type    = lambda p: get_type(p),
            lines   = lambda p: with_open(lambda f: f.read().splitlines(), p),
            use_record_class=False
        )
    )
    >> map(
        rec.update(
            parsed_rows = lambda r: [
                dict(zip(column_names, parse_fixed_width_unfold(line, column_widths))) | {
                    "Course": r["course"],
                    "Section": r["section"],
                    "Type": r["type"]
                }
                for line in r["lines"] if line.strip()
            ]
        )
    )
    >> fold(lambda acc, r: acc + r["parsed_rows"], [])
)


# --- Save to CSV ---

In [37]:
# --- Save to CSV ---
output_path = "combined_outputP4.csv"
output_columns = column_names + ["Course", "Section", "Type"]

with open(output_path, mode="w", newline='', encoding="utf-8") as out_file:
    writer = csv.DictWriter(out_file, fieldnames=output_columns)
    writer.writeheader()
    writer.writerows(combined_records)

print(f"✅ Combined CSV saved as: {output_path}")

✅ Combined CSV saved as: combined_outputP4.csv
